In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation,Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from numpy import array

In [7]:
df = pd.read_csv("/home/wojtek/Desktop/emotion_recognition/dataset/internet_dataset/final_dataset_v8_top_3000_spellchecked.csv", sep=',')

In [8]:
df.head()

,emotion,comment
0,happiness,period fall love time meet especially not meet...
1,fear,involve traffic accident
2,anger,drive home several day hard work motorist ahea...
3,sadness,lose person meant
4,disgust,time knock deer sight animal injury helplessne...


In [9]:
X = df['comment']
y = df['emotion']

In [10]:
y.value_counts()

happiness    3000
fear         3000
anger        3000
sadness      3000
disgust      3000
surprise     2999
Name: emotion, dtype: int64

In [11]:
text = df['comment'].tolist()

In [12]:
token = Tokenizer()
token.fit_on_texts(text)

In [13]:
vocab_size = len(token.word_index) + 1

In [14]:
vocab_size

19097

In [15]:
# token.index_word

In [16]:
encoded_text = token.texts_to_sequences(text)
encoded_text[:3]

[[814, 360, 30, 10, 122, 529, 1, 122, 82, 10],
 [678, 896, 320],
 [216,
  31,
  306,
  6,
  175,
  15,
  9237,
  963,
  216,
  269,
  126,
  399,
  633,
  447,
  1442,
  93,
  2760]]

In [17]:
max_length = 120 # machine learning potrzebuje żeby zdania były max sized (pewnie pozostałe to 0)
X = pad_sequences(encoded_text, maxlen=max_length, padding='post') # padding post po to, by 0 byly dodane na koncu listy
X

array([[  814,   360,    30, ...,     0,     0,     0],
       [  678,   896,   320, ...,     0,     0,     0],
       [  216,    31,   306, ...,     0,     0,     0],
       ...,
       [19096,    20,   395, ...,     0,     0,     0],
       [  192,  1144,    73, ...,     0,     0,     0],
       [  100,   509,  3340, ...,     0,     0,     0]], dtype=int32)

In [18]:
X.shape

(17999, 120)

In [19]:
###`GloVe` Vectors

In [20]:
path_to_glove_file = "/home/wojtek/Desktop/glove.6B.100d.txt"

In [21]:
glove_vectors = dict()

In [22]:
%%time
with open(path_to_glove_file) as file:
    for line in file:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:])  # list will be converted as array
        glove_vectors[word] = vectors

CPU times: user 6.06 s, sys: 3.05 s, total: 9.11 s
Wall time: 10.2 s


In [23]:
len(glove_vectors.keys())

400000

In [24]:
glove_vectors.get('you').shape

(100,)

In [25]:
word_vector_matrix = np.zeros((vocab_size, 100))

In [26]:
for word, index in token.word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
    # else:
        # print(word)

In [27]:
### TF2.0 and Keras Model Building

In [28]:
X

array([[  814,   360,    30, ...,     0,     0,     0],
       [  678,   896,   320, ...,     0,     0,     0],
       [  216,    31,   306, ...,     0,     0,     0],
       ...,
       [19096,    20,   395, ...,     0,     0,     0],
       [  192,  1144,    73, ...,     0,     0,     0],
       [  100,   509,  3340, ...,     0,     0,     0]], dtype=int32)

In [29]:
y

0        happiness
1             fear
2            anger
3          sadness
4          disgust
           ...    
17994     surprise
17995     surprise
17996     surprise
17997     surprise
17998     surprise
Name: emotion, Length: 17999, dtype: object

In [30]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [74]:
le = LabelEncoder()
y_label = le.fit_transform(y)
y_label = to_categorical(y_label, num_classes=6)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, random_state=42, test_size=0.2)

In [33]:
y_train.shape

(14399, 6)

In [34]:
X_train.shape

(14399, 120)

In [35]:
vec_size = 100

In [36]:
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights=[word_vector_matrix], trainable=False)) # wagi to sa wektory

model.add(Conv1D(64, 8, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
    
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(6, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=60, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/60
1200/1200 [==============================] - 24s 20ms/step - loss: 1.8111 - accuracy: 0.2022 - val_loss: 1.7356 - val_accuracy: 0.2581
Epoch 2/60
1200/1200 [==============================] - 24s 20ms/step - loss: 1.7141 - accuracy: 0.2466 - val_loss: 1.6662 - val_accuracy: 0.3119
Epoch 3/60
1200/1200 [==============================] - 27s 23ms/step - loss: 1.6499 - accuracy: 0.2770 - val_loss: 1.5937 - val_accuracy: 0.3506
Epoch 4/60
1200/1200 [==============================] - 28s 23ms/step - loss: 1.5990 - accuracy: 0.3066 - val_loss: 1.5366 - val_accuracy: 0.3853
Epoch 5/60
1200/1200 [==============================] - 26s 22ms/step - loss: 1.5560 - accuracy: 0.3275 - val_loss: 1.5029 - val_accuracy: 0.4231
Epoch 6/60
1200/1200 [==============================] - 25s 20ms/step - loss: 1.5183 - accuracy: 0.3636 - val_loss: 1.4579 - val_accuracy: 0.4425
Epoch 7/60
1200/1200 [==============================] - 25s 21ms/step - loss: 1.4849 - accuracy: 0.3756 - val_loss: 1.4107 -

In [37]:
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights=[word_vector_matrix], trainable=False)) # wagi to sa wektory
model.add(LSTM(128, recurrent_dropout=0.5, dropout=0.5))
model.add(Dense(6))
model.add(Activation('softmax'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 100)          1909700   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 6)                 774       
_________________________________________________________________
activation (Activation)      (None, 6)                 0         
Total params: 2,027,722
Trainable params: 118,022
Non-trainable params: 1,909,700
_________________________________________________________________


In [39]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/30
450/450 [==============================] - 112s 245ms/step - loss: 1.7924 - accuracy: 0.1636 - val_loss: 1.7918 - val_accuracy: 0.1714
Epoch 2/30
450/450 [==============================] - 112s 250ms/step - loss: 1.7923 - accuracy: 0.1642 - val_loss: 1.7921 - val_accuracy: 0.1700
Epoch 3/30
450/450 [==============================] - 109s 242ms/step - loss: 1.7922 - accuracy: 0.1611 - val_loss: 1.7926 - val_accuracy: 0.1653
Epoch 4/30
450/450 [==============================] - 113s 252ms/step - loss: 1.7923 - accuracy: 0.1672 - val_loss: 1.7920 - val_accuracy: 0.1600
Epoch 5/30
450/450 [==============================] - 113s 250ms/step - loss: 1.7919 - accuracy: 0.1652 - val_loss: 1.7929 - val_accuracy: 0.1600
Epoch 6/30
108/450 [======>.......................] - ETA: 1:33 - loss: 1.7924 - accuracy: 0.1716

KeyboardInterrupt: 

In [95]:
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights=[word_vector_matrix], trainable=False)) # wagi to sa wektory

model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(5))
# model.add(Dropout(0.5))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(5))
# model.add(Dropout(0.5))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
# model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
    
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
# model.add(GlobalMaxPooling1D())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/5
225/225 [==============================] - 13s 55ms/step - loss: 1.6486 - accuracy: 0.2565 - val_loss: 1.4697 - val_accuracy: 0.3336
Epoch 2/5
225/225 [==============================] - 12s 55ms/step - loss: 1.3906 - accuracy: 0.3882 - val_loss: 1.2728 - val_accuracy: 0.4653
Epoch 3/5
225/225 [==============================] - 12s 55ms/step - loss: 1.2275 - accuracy: 0.4725 - val_loss: 1.1771 - val_accuracy: 0.5053
Epoch 4/5
225/225 [==============================] - 10s 46ms/step - loss: 1.1165 - accuracy: 0.5364 - val_loss: 1.1605 - val_accuracy: 0.5200
Epoch 5/5
225/225 [==============================] - 11s 48ms/step - loss: 1.0393 - accuracy: 0.5701 - val_loss: 1.1533 - val_accuracy: 0.5472


In [156]:
pred = model.predict(X_test[1:2])

In [157]:
pred

array([[0.03753073, 0.01475197, 0.20454566, 0.30121052, 0.23221147,
        0.20974961]], dtype=float32)

In [158]:
pred[0:1]

array([[0.03753073, 0.01475197, 0.20454566, 0.30121052, 0.23221147,
        0.20974961]], dtype=float32)

In [159]:
np.argmax(pred[0:1], axis=None, out=None)

3

In [160]:
result = (-pred).argsort()[:2]

In [161]:
top_prediction, secondary_prediction = result[0][0], result[0][1]

In [162]:
top_prediction

3

In [134]:
y_test[1:2]

array([[0., 0., 0., 0., 0., 1.]], dtype=float32)

In [135]:
np.argmax(y_test[0:1], axis=None, out=None)

0

In [136]:
le.inverse_transform((np.argmax(y_test[0:1], axis=None, out=None),))[0]

'anger'

In [137]:
le.inverse_transform((np.argmax(pred[0:1], axis=None, out=None),))[0]

'anger'

array([], shape=(0, 6), dtype=float32)

In [100]:
pred[0][np.argmax(pred[0:1], axis=None, out=None)]

0.9998349

In [101]:
len(y_test)

3600

In [106]:
X_test[0:1][0]

array([ 105,   17,    1,  110,  998,  473,  283,  710,   44, 1291,   18,
        555,   24,  177,   47,  555,   21,  163,    4,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [109]:
tokenizer.sequences_to_texts(X_test[0:1][0])

NameError: name 'tokenizer' is not defined

In [110]:
reverse_word_map = dict(map(reversed, token.word_index.items()))

In [115]:
sentence = []
for i in X_test[0:1][0]:
    if i == 0:
        break
    sentence.append(reverse_word_map[i])
' '.join(sentence)

'parent think not best pas final examination secondary school achieve good grade say nothing bad grade would become angry'

In [170]:
# for i in range(len(y_test)+1):
for i in range(200):
    pred = model.predict(X_test[i:i+1])
    another_pred = (-pred).argsort()[:2]
    pred = le.inverse_transform((np.argmax(pred, axis=None, out=None),))[0]
    result = le.inverse_transform((np.argmax(y_test[i:i+1], axis=None, out=None),))[0]
    prim_pred = le.inverse_transform((another_pred[0][0],))[0]
    sec_pred = le.inverse_transform((another_pred[0][1],))[0]
    if pred != result:
        sentence = []
        for a in X_test[i:i+1][0]:
            if a == 0:
                break
            sentence.append(reverse_word_map[a])
        classified_sentence = ' '.join(sentence)
        print("sentence")
        print(classified_sentence)
        print("result")
        print(result)
        print("prediction")
        print(pred)
        print("primary prediction")
        print(prim_pred)
        print("secondary prediction")
        print(sec_pred)
        print("----")

sentence
trivia work no computer
result
surprise
primary prediction
happiness
secondary prediction
sadness
----
sentence
yeah well sorry not believe kiss sa
result
surprise
primary prediction
sadness
secondary prediction
fear
----
sentence
friend promise would definitely call tell visit not
result
anger
primary prediction
happiness
secondary prediction
sadness
----
sentence
watch series call dynasty detest
result
disgust
primary prediction
fear
secondary prediction
sadness
----
sentence
sorry suck know ve yay half day though
result
fear
primary prediction
happiness
secondary prediction
surprise
----
sentence
new project go incredibly well not tire today think get monday permanently
result
happiness
primary prediction
sadness
secondary prediction
fear
----
sentence
think haircut isnt bad look yesterday still bad tho
result
surprise
primary prediction
sadness
secondary prediction
fear
----
sentence
alive go notebook all make cry stuff
result
surprise
primary prediction
sadness
secondary 

In [128]:
le.inverse_transform((np.argmax(pred[0:1], axis=None, out=None),))[0]

'anger'

In [130]:
pred

'surprise'

In [175]:
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights=[word_vector_matrix], trainable=False)) # wagi to sa wektory

model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(5))

model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
    
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(6, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=60, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/60
450/450 [==============================] - 12s 25ms/step - loss: 1.8033 - accuracy: 0.1981 - val_loss: 1.7596 - val_accuracy: 0.2367
Epoch 2/60
450/450 [==============================] - 11s 24ms/step - loss: 1.7447 - accuracy: 0.2418 - val_loss: 1.6967 - val_accuracy: 0.2622
Epoch 3/60
450/450 [==============================] - 11s 24ms/step - loss: 1.6735 - accuracy: 0.2719 - val_loss: 1.6112 - val_accuracy: 0.2972
Epoch 4/60
450/450 [==============================] - 11s 25ms/step - loss: 1.5909 - accuracy: 0.3079 - val_loss: 1.5329 - val_accuracy: 0.3483
Epoch 5/60
450/450 [==============================] - 11s 25ms/step - loss: 1.5151 - accuracy: 0.3475 - val_loss: 1.4525 - val_accuracy: 0.3886
Epoch 6/60
450/450 [==============================] - 11s 25ms/step - loss: 1.4500 - accuracy: 0.3832 - val_loss: 1.3802 - val_accuracy: 0.4317
Epoch 7/60
450/450 [==============================] - 12s 26ms/step - loss: 1.3755 - accuracy: 0.4306 - val_loss: 1.3165 - val_accuracy: